In [1]:
import numpy as np
from scipy import optimize
from capsolpy import defcapsol as cs
# reload(capsolpy)

# Develop CapSolPy

- `cs.generategrid` currently works
- `cs.setupprobe` does not (seg faults on MAC OS X, even though it works in fortran)
- `cs.setupprobe` is relatively simple - we could translate that code to python

In [2]:
# Parameters
nuni = 1
n = 500
m = 500
h0 = 0.5
rho_max = 1e6
z_max = 1e6
z = 10.0
l_js = 20
l = l_js + int(np.round(z/h0))
verbose=0
hsam = 10.0
rtip = 20.0
theta = 15 * 180 / np.pi 
hcone = 15000.0
rcant = 500.0
dcant = 15000.0
eps_r = 3.0


r, hn, hm, zm = cs.generategrid(nuni, h0, rho_max,z_max,z,hsam,n,m,l,verbose)

In [3]:
print(cs.generategrid.__doc__)

r,hn,hm,zm = generategrid(nuni,h0,rho_max,z_max,z,hsam,n,m,l,verbose)

Wrapper for ``generategrid``.

Parameters
----------
nuni : input int
h0 : input float
rho_max : input float
z_max : input float
z : input float
hsam : input float
n : input int
m : input int
l : input int
verbose : input int

Returns
-------
r : rank-1 array('f') with bounds (n + 1)
hn : rank-1 array('f') with bounds (n + 1)
hm : rank-1 array('f') with bounds (m+l+1)
zm : rank-1 array('f') with bounds (m+l+1)



In [ ]:
def geometric_sum(r, a1, n):
    return sum(np.array([a1 * r ** i for i in range(n)]))

def find_ratio(total, h0, terms):
    r = 1.001
    s = 0
    while s < total:
        s = geometric_sum(r, h0, terms)
        r += 0.001
    return r

def radial_grid(nuni, n, h0, rho_max):
    r = np.zeros(n+1)
    dr = np.zeros(n) # Could also use n+1
    dr[:nuni] = h0
    r[1:nuni+1] = np.cumsum(dr[:nuni])
    r_left = rho_max - r[nuni+1] 
    return r, dr

In [ ]:
def approx_poly(n, s, a):
    return np.roots([((-2 + n)*(-1 + n)*n)/6., ((-1.0 + n)*n)/2., n-s/a])

- `r` is the radii (`n` + 1 points spaced from 0 to `rho_max`)
- `hn` is the spacing between radii